In [2]:
import pandas as pd
import numpy as np
from data_prep import features, targets, features_test, targets_test

import string  
import re
import scipy
import matplotlib.pyplot as plt
import urllib 
import sklearn
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras.layers import Dense, Dropout, Flatten

import tensorflow.keras.backend as K

from gensim.models import Word2Vec, KeyedVectors

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
display(train)
display(test)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [3]:
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

test['keyword'].fillna('no keyword', inplace = True) 
test['keyword'] = test['keyword'].str.replace('%20', ' ')
test['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()

test['length'] = test['text'].str.len()

train.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


In [4]:
#limpiando el texto

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

test['clean_text'] = test['text'].str.lower()

test['clean_text'] = test['clean_text'].apply(only_letters) 

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

test['clean_text'] = test['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

test['clean_text'] = test['clean_text'].apply(filter_stopwords) 


#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)

test['clean_text'] = test['clean_text'].apply(lemmatize_tweet)  

test['clean_text'] = test['clean_text'].apply(transform_to_text)

display(train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [5]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, MaxPooling1D


In [239]:
clean_text = train['clean_text'].values
labels = train['target'].values
X_train, X_test, y_train, y_test = train_test_split(clean_text, labels, test_size=0.20, random_state=15)
X_set_test = test['clean_text'].values

In [240]:
#Tokenizador
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1
vocab_size*300

4017600

In [241]:
# A secuencia de enteros
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_set_test = tokenizer.texts_to_sequences(X_set_test)

In [242]:
# Padding al final
maxlen = 20
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
X_set_test = pad_sequences(X_set_test, padding='post', maxlen=maxlen)

In [10]:
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [11]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

    
del(word_vectors)

In [243]:
input_shape = (X_train.shape[0],X_train.shape[1])
model = keras.Sequential()

embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(Conv1D(515, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [244]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [245]:
print(model.summary())

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 20, 300)           4017600   
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 19, 515)           309515    
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 6, 515)            0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 3090)              0         
_________________________________________________________________
dense_50 (Dense)             (None, 15)                46365     
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 16        
Total params: 4,373,496
Trainable params: 355,896
Non-trainable params: 4,017,600
_____________________________________

In [246]:
history = model.fit(X_train, y_train, batch_size=64, epochs=3, verbose=1, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

Train on 4872 samples, validate on 1218 samples
Epoch 1/3
4872/4872 [==============================] - 7s 1ms/sample - loss: 0.5216 - accuracy: 0.7506 - val_loss: 0.4299 - val_accuracy: 0.8153
Epoch 2/3
4872/4872 [==============================] - 6s 1ms/sample - loss: 0.3722 - accuracy: 0.8432 - val_loss: 0.4171 - val_accuracy: 0.8194
Epoch 3/3
4872/4872 [==============================] - 6s 1ms/sample - loss: 0.2744 - accuracy: 0.8945 - val_loss: 0.4579 - val_accuracy: 0.8030
1523/1 [=============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [247]:
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

Loss: 0.4740680388651314
Accuracy: 0.8017071485519409


In [248]:
#Set de test
preds = model.predict(X_set_test)
preds = (preds >= 0.5).astype(int)
preds

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [249]:

test['target'] = preds
submission = test.loc[:,['id','target']]
submission.to_csv(path_or_buf='red_convolucional_word2vec_5.csv',header=True,index=False)